In [1]:
import sys
sys.path.insert(0, '../')
import myTA
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np

from strategies import Backtesting, MACDZeroHistPredictionAndSMA, makeAnalysis, SharpeRatio, get_df, init_backtesting_results_file, get_allowedIntervals, get_parameter_combination_number

c:\Users\QbaSo\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
sharpe_ratio = SharpeRatio()
allowedIntervals = get_allowedIntervals()
init_backtesting_results_file("backtest_resluts/backtesting_results.csv")

In [3]:
tickers = ['AAPL']#, 'AMZN', 'SPY', 'TSLA', 'usdeur=x']

linerPredictionWindows = [4]
fastperiods = [12]
slowperiods = [26]
signalperiods = [9]

# linerPredictionWindows = np.arange(2,6)
# fastperiods = np.arange(8,18)
# slowperiods = np.arange(20,30)
# signalperiods = np.arange(5,13)

df = pd.DataFrame()

get_parameter_combination_number(tickers, linerPredictionWindows, fastperiods, slowperiods, signalperiods, allowedIntervals)
for i in range(len(allowedIntervals)):
    end = dt.datetime.now()
    start = end - allowedIntervals.iloc[i]['back_allowed']
    interval = allowedIntervals.iloc[i]['intervals']

    for ticker in tickers:
        data = yf.download(ticker, start=start, end=end, interval=interval, progress=False)

        for linerPredictionWindow in linerPredictionWindows:
            for fastperiod in fastperiods:
                for slowperiod in slowperiods:
                    for signalperiod in signalperiods:
                        strategy = MACDZeroHistPredictionAndSMA(linerPredictionWindow, ticker, start, end, interval, fastperiod, slowperiod, signalperiod)
                        data = makeAnalysis(strategy, data)
                        strategy.getBackTestSignals(data)

                        bt = Backtesting(1000, strategy)
                        bt.calculateProfit()
                        df = get_df(df, bt, sharpe_ratio)

    df.to_csv('backtest_resluts/backtesting_results.csv', mode='a', index=False, header=False)
    df = pd.DataFrame()

backteste will run for: 8 pameters combination


In [4]:
results = pd.read_csv("backtest_resluts/backtesting_results.csv")
results

,interval,start,end,ticker,linerPredictionWindow,fastperiod,slowperiod,signalperiod,budget_start,result,sharpe_ratio,ROI
0,1m,2023-02-24 21:32:41.323569,2023-03-01 21:32:41.323569,AAPL,4,12,26,9,1000,-3.0,-27.69,-0.3
1,2m,2023-01-01 21:32:41.829986,2023-03-01 21:32:41.829986,AAPL,4,12,26,9,1000,-8.0,-1.11,-0.8
2,5m,2023-01-01 21:32:44.064023,2023-03-01 21:32:44.064023,AAPL,4,12,26,9,1000,19.0,2.07,1.9
3,15m,2023-01-01 21:32:45.100543,2023-03-01 21:32:45.100543,AAPL,4,12,26,9,1000,20.0,2.79,2.0
4,30m,2023-01-01 21:32:45.711540,2023-03-01 21:32:45.711540,AAPL,4,12,26,9,1000,17.0,2.75,1.7
5,1h,2021-03-02 21:32:46.026538,2023-03-01 21:32:46.026538,AAPL,4,12,26,9,1000,40.0,0.27,4.0
6,90m,2023-01-01 21:32:47.608575,2023-03-01 21:32:47.608575,AAPL,4,12,26,9,1000,4.0,3.00,0.4
7,1d,1971-03-14 21:32:47.862577,2023-03-01 21:32:47.862577,AAPL,4,12,26,9,1000,342.0,0.44,34.2
